In [ ]:
import numpy as np
import pandas as pd
import ast
import nltk

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.head(1)

In [ ]:
movies = movies[['movie_id', 'genres', 'keywords', 'title', 'cast', 'crew', 'overview' ]]
# budget
# genres
# id
# keywords
# orignal_title
# overview
# runtime
# tagline
# cast
# crew


# now we need to merge the genres, cast ,crew column to form a tags column
# we will take only top three casts and only the director from the crew.

# this is the part of data preprocessing






In [ ]:
movies


In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

# we have to convert this format in , 
#  ['action', 'adventure', 'fantasy', 'science fiction']

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
convert( '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
)

In [ ]:
movies['genres'].apply(convert)

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()


In [ ]:
movies.iloc[0].keywords

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies

In [ ]:
movies['cast'][0]

In [ ]:
def convert3(obj):
    cnt =0
    L = []
    for i in ast.literal_eval(obj):
        if cnt <= 3:
          L.append(i['name'])
          cnt++1
        else : 
          break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['crew'][3]



In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
          L.append(i['name'])
          break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies

In [ ]:
movies['overview'][0]
# this overview section is a string, so we need to convert it in a list so that we can concatenate it with other lists.


In [ ]:
# this converts text in list

nltk.download('punkt')
from nltk.tokenize import word_tokenize
movies = movies.dropna(subset=['overview'])

movies['overview'] = movies['overview'].apply(word_tokenize)


In [ ]:
# this is a check mark

for index, tags in enumerate(movies['overview']):
    if not isinstance(tags, list):
        print(f"Element at index {index} is not a list. Type: {type(tags)}")
    else:
        print(f"Element at index {index} is a list. Type: {type(tags)}")
        

In [ ]:
movies['overview'][0]

In [ ]:
# we need to remove spaces between the same name 
movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])



In [ ]:
movies['tags'] = movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['genres']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
new_df

In [ ]:
# we need to convert the list into string format again " ".join(x) for i in x]


new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

# if u run above statement multiple times it will add a space that number of time


In [ ]:
new_df['tags'][0]


In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
# this is recommended to convert all text into lower case by this function

In [ ]:
# now we have a fine clear data, now we have to vectorize the data
'''
VECTORIZATION

this is the process of converting text into vectors. then the vectors closest to a particular vector will be the best match for that particular vector.
                      Vectorisation
                       /    |     \
                      /     |      \
                     /      |       \
            bag of words   
'''

In [ ]:
# now there are some words like (acted, acting, actors, actor) ... these all are similar so we need to replace this by one
# this process is called stamming

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features= 5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()


In [ ]:
vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
# new_df['title'] == 'Avatar' # this checks the index of movie in data
movie_index = new_df[new_df['title'] == 'Avatar'].index[0]
movie_index

In [ ]:
'''
1. enumerate is like map, it convert the list into tuple to store the index.
2. Reverse is to arrange in ascending order
3. [1:6] return the top five matching movies and x[1] sort on the basis of 1st index
'''
# sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x:x[1:6])

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]    # is the distance array of movies from movie at this index
    movies_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('1982')

In [ ]:
# new_df.iloc[1216][1] #actually here index 1 is 'title' of the movie

In [ ]:
import pickle

In [ ]:
movies_list = pickle.dump(new_df,open('movies.pkl', 'wb'))  # wb stands for write binary mode

In [ ]:
movies = new_df['title']  # this will fetch the name of all movies
movies

In [ ]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))

In [1]:
# Loading raw german data
german_df = pd.read_csv("europarl-v7.de-en.de", "utf-8", header=None, names=["German"])
german_df.head()

NameError: name 'pd' is not defined